# 03 - Model Serving with KServe

## Overview

This notebook deploys the trained model as a **KServe InferenceService** for real-time predictions. The key feature is **Feast integration** - clients send only entity IDs, and the server fetches features from Feast's online store.

### What You'll Learn

| Step | Action | Component |
|------|--------|-----------|
| 1 | Create serving script | ConfigMap with KServe Model class |
| 2 | Deploy InferenceService | KServe with Feast integration |
| 3 | Make predictions | REST API (V2 protocol) |

### Inference Flow

```
┌─────────────────────────────────────────────────────────────────────────────┐
│                        INFERENCE DATA FLOW                                  │
├─────────────────────────────────────────────────────────────────────────────┤
│                                                                             │
│  Client                    KServe                   Feast Online Store      │
│  ┌─────────────────┐      ┌─────────────────┐      ┌─────────────────┐      │
│  │ {store_id: 1,   │      │                 │      │                 │      │
│  │  dept_id: 3}    │─────▶│  1. Receive     │─────▶│  2. Fetch       │      │
│  │                 │      │     request     │      │     features    │      │
│  └─────────────────┘      │                 │◀─────│     (<50ms)     │      │
│                           │  3. Scale       │      └─────────────────┘      │
│                           │     features    │                               │
│                           │                 │                               │
│  ┌─────────────────┐      │  4. Model       │                               │
│  │                 │◀─────│     inference   │                               │
│  │ {prediction:    │      │                 │                               │
│  │  $96,763}       │      │  5. Inverse     │                               │
│  │                 │      │     transform   │                               │
│  └─────────────────┘      └─────────────────┘                               │
│                                                                             │
└─────────────────────────────────────────────────────────────────────────────┘
```

### Why Feast Integration?

| Approach | Client Sends | Pros | Cons |
|----------|--------------|------|------|
| **Without Feast** | All 21 features | Simple server | Feature drift, complex client |
| **With Feast** | Entity IDs only | No drift, simple client | Requires Feast online store |

**Key Benefit**: Training and serving use the **same FeatureService**, eliminating feature skew.

### Prerequisites

1. `01-feast-features.ipynb` completed (online store populated)
2. `02-training.ipynb` completed (model artifacts saved)
3. KServe operator installed on cluster

---
## Step 0: Install Dependencies

In [ ]:
%pip install -q kserve pandas requests kubernetes

In [ ]:
from kserve import KServeClient, V1beta1InferenceService, V1beta1InferenceServiceSpec, V1beta1PredictorSpec
from kubernetes import client as k8s_client, config
from kubernetes.client import V1Container, V1ResourceRequirements, V1VolumeMount, V1Volume
from kubernetes.client import V1PersistentVolumeClaimVolumeSource, V1ConfigMapVolumeSource
from kubernetes.client import V1EnvVar, V1Probe, V1HTTPGetAction, V1ConfigMap, V1ObjectMeta
import pandas as pd
import requests
import time
import json
from datetime import datetime

---
## Step 1: Configuration

| Parameter | Value | Purpose |
|-----------|-------|---------|
| `MODEL_NAME` | sales-forecast | InferenceService name |
| `MODEL_DIR` | /shared/models | Path to model artifacts |
| `FEAST_SERVER_URL` | feast-server:6566 | Feast online feature server |

In [ ]:
NAMESPACE = "feast-trainer-demo"
MODEL_NAME = "sales-forecast"
MODEL_DIR = "/shared/models"
PVC_NAME = "shared"
CONFIGMAP_NAME = "sales-forecast-serve"

# Trainer image (has PyTorch pre-installed, matches Ray cluster Python version)
TRAINER_IMAGE = "quay.io/modh/training:py312-cuda128-torch280"

# Feast Feature Server (deployed via manifests)
FEAST_SERVER_URL = f"http://feast-server.{NAMESPACE}.svc.cluster.local:6566"

print(f"📋 Configuration:")
print(f"   Namespace: {NAMESPACE}")
print(f"   Model: {MODEL_NAME}")
print(f"   Image: {TRAINER_IMAGE}")
print(f"   PVC: {PVC_NAME}")
print(f"   Feast Server: {FEAST_SERVER_URL}")

---
## Step 2: Create Serving Script

The serving script implements KServe's `Model` interface:

| Method | Purpose | Called When |
|--------|---------|-------------|
| `load()` | Load model, scalers, feature columns | Server startup |
| `preprocess()` | Extract entities → Call Feast → Build feature matrix | Each request |
| `predict()` | Scale features → Model inference → Inverse transform | Each request |
| `postprocess()` | Format as V2 InferResponse | Each request |

### Request Flow

```python
# Input: Entity IDs
{"inputs": [{"name": "entities", "data": [{"store_id": 1, "dept_id": 3}]}]}

# preprocess() → Feast lookup → Feature matrix
[[lag_1, lag_2, ..., temperature, fuel_price]]  # 21 features

# predict() → Model inference → Inverse transform
[96763.45]  # Predicted weekly sales

# Output: V2 Response
{"outputs": [{"name": "predictions", "data": [96763.45]}]}
```

In [ ]:
SERVE_SCRIPT = '''#!/usr/bin/env python3
"""Sales Forecasting Inference Server (KServe V2 Protocol)

This server:
1. Receives entity IDs (store_id, dept_id)
2. Fetches features from Feast online store
3. Runs model inference
4. Returns predicted weekly sales
"""
import os
import json
import requests
import torch
import torch.nn as nn
import joblib
import numpy as np
import logging
from typing import Dict, Union
from kserve import Model, ModelServer, InferRequest, InferResponse, InferOutput

logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(message)s")
logger = logging.getLogger(__name__)

FEAST_SERVER_URL = os.getenv("FEAST_SERVER_URL", "http://feast-server.feast-trainer-demo.svc.cluster.local:6566")


class SalesMLP(nn.Module):
    """Multi-layer perceptron matching training architecture."""
    
    def __init__(self, input_dim, hidden_dims=None, dropout=0.2):
        super().__init__()
        if hidden_dims is None:
            hidden_dims = [256, 128, 64]
        layers = []
        prev_dim = input_dim
        for h_dim in hidden_dims:
            layers.extend([
                nn.Linear(prev_dim, h_dim),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.Dropout(dropout)
            ])
            prev_dim = h_dim
        layers.append(nn.Linear(prev_dim, 1))
        self.net = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.net(x).squeeze(-1)


class SalesForecastModel(Model):
    """KServe Model with Feast feature lookup."""
    
    def __init__(self, name: str):
        super().__init__(name)
        self.model = None
        self.scalers = None
        self.feature_cols = None
        self.use_log_transform = False
        self.ready = False
    
    def load(self):
        """Load model and scalers from shared storage."""
        model_dir = os.getenv("MODEL_DIR", "/shared/models")
        logger.info(f"Loading model from {model_dir}...")
        
        # Load metadata
        with open(f"{model_dir}/model_metadata.json") as f:
            metadata = json.load(f)
        
        # Initialize model with same architecture
        hidden_dims = metadata.get("hidden_dims", [256, 128, 64])
        dropout = metadata.get("dropout", 0.2)
        input_dim = metadata["input_dim"]
        
        self.model = SalesMLP(input_dim, hidden_dims, dropout)
        self.model.load_state_dict(
            torch.load(f"{model_dir}/model_best.pt", map_location="cpu", weights_only=True)
        )
        self.model.eval()
        
        # Load scalers
        self.scalers = joblib.load(f"{model_dir}/scalers.joblib")
        self.feature_cols = metadata["feature_columns"]
        self.use_log_transform = self.scalers.get("use_log_transform", False)
        
        logger.info(f"Model loaded: {len(self.feature_cols)} features")
        self.ready = True
    
    def _get_features_from_feast(self, entity_rows):
        """Fetch features from Feast online store."""
        payload = {
            "feature_service": "inference_features",
            "entities": {
                "store_id": [row["store_id"] for row in entity_rows],
                "dept_id": [row["dept_id"] for row in entity_rows],
            }
        }
        
        response = requests.post(
            f"{FEAST_SERVER_URL}/get-online-features",
            json=payload,
            timeout=10
        )
        response.raise_for_status()
        result = response.json()
        
        # Parse response into feature rows
        feature_names = result.get("metadata", {}).get("feature_names", [])
        results = result.get("results", [])
        
        features = []
        for i in range(len(entity_rows)):
            row = {}
            for j, name in enumerate(feature_names):
                if name in self.feature_cols and j < len(results):
                    val = results[j].get("values", [None])[i] if i < len(results[j].get("values", [])) else None
                    row[name] = val if val is not None else 0
            features.append(row)
        
        return features
    
    def preprocess(self, payload: Union[Dict, InferRequest], headers: Dict = None) -> np.ndarray:
        """Extract entities and fetch features from Feast."""
        if isinstance(payload, InferRequest):
            inputs = {inp.name: inp.data for inp in payload.inputs}
        else:
            inputs = {inp["name"]: inp.get("data") for inp in payload.get("inputs", [])}
        
        # Option 1: Client sends entity IDs → fetch from Feast
        if "entities" in inputs:
            entities = inputs["entities"]
            if isinstance(entities, list) and len(entities) > 0:
                feature_rows = self._get_features_from_feast(entities)
                X = np.array(
                    [[row.get(c, 0) for c in self.feature_cols] for row in feature_rows],
                    dtype=np.float32
                )
                return X
        
        # Option 2: Client sends pre-computed features directly
        if "features" in inputs:
            return np.array(inputs["features"], dtype=np.float32)
        
        raise ValueError("Input must have \'entities\' or \'features\'")
    
    def predict(self, X: np.ndarray, headers: Dict = None) -> np.ndarray:
        """Run model inference."""
        X_scaled = self.scalers["scaler_X"].transform(X)
        
        with torch.no_grad():
            preds = self.model(torch.FloatTensor(X_scaled)).numpy()
        
        # Inverse transform
        predictions = self.scalers["scaler_y"].inverse_transform(preds.reshape(-1, 1)).flatten()
        if self.use_log_transform:
            predictions = np.expm1(predictions)
        
        return predictions
    
    def postprocess(self, predictions: np.ndarray, headers: Dict = None) -> Union[Dict, InferResponse]:
        """Format as V2 InferResponse."""
        return InferResponse(
            model_name=self.name,
            response_id=str(hash(str(predictions.tolist()))),
            infer_outputs=[
                InferOutput(
                    name="predictions",
                    shape=list(predictions.shape),
                    datatype="FP32",
                    data=predictions.tolist()
                )
            ]
        )


if __name__ == "__main__":
    model = SalesForecastModel("sales-forecast")
    model.load()
    ModelServer().start([model])
'''

In [ ]:
# Load Kubernetes config
try:
    config.load_incluster_config()
    print("✅ Loaded in-cluster Kubernetes config")
except:
    config.load_kube_config()
    print("✅ Loaded local Kubernetes config")

core_v1 = k8s_client.CoreV1Api()

In [ ]:
# Create ConfigMap with serving script
configmap = V1ConfigMap(
    metadata=V1ObjectMeta(
        name=CONFIGMAP_NAME,
        namespace=NAMESPACE,
        labels={"app": "sales-forecasting"}
    ),
    data={"serve.py": SERVE_SCRIPT}
)

try:
    core_v1.read_namespaced_config_map(CONFIGMAP_NAME, NAMESPACE)
    print(f"⚠️ ConfigMap '{CONFIGMAP_NAME}' exists, replacing...")
    core_v1.replace_namespaced_config_map(CONFIGMAP_NAME, NAMESPACE, configmap)
except:
    print(f"📦 Creating ConfigMap '{CONFIGMAP_NAME}'...")
    core_v1.create_namespaced_config_map(NAMESPACE, configmap)

print(f"✅ ConfigMap created with serve.py script")

---
## Step 3: Deploy InferenceService

### InferenceService Configuration

| Setting | Value | Purpose |
|---------|-------|---------|
| `minReplicas` | 1 | Always have at least 1 replica |
| `maxReplicas` | 3 | Scale up under load |
| `deploymentMode` | RawDeployment | Use standard K8s deployment |
| `readinessProbe` | /v2/health/ready | Health check endpoint |

In [ ]:
kserve_client = KServeClient()
print(f"✅ KServeClient initialized")

In [ ]:
# Build InferenceService spec
isvc = V1beta1InferenceService(
    api_version="serving.kserve.io/v1beta1",
    kind="InferenceService",
    metadata=k8s_client.V1ObjectMeta(
        name=MODEL_NAME,
        namespace=NAMESPACE,
        labels={"app": "sales-forecasting"},
        annotations={"serving.kserve.io/deploymentMode": "RawDeployment"}
    ),
    spec=V1beta1InferenceServiceSpec(
        predictor=V1beta1PredictorSpec(
            min_replicas=1,
            max_replicas=3,
            containers=[
                V1Container(
                    name="kserve-container",
                    image=TRAINER_IMAGE,
                    command=["/bin/bash", "-c"],
                    args=["pip install -q kserve joblib numpy scikit-learn && python /scripts/serve.py"],
                    ports=[k8s_client.V1ContainerPort(container_port=8080, protocol="TCP")],
                    env=[
                        V1EnvVar(name="MODEL_DIR", value=MODEL_DIR),
                        V1EnvVar(name="FEAST_SERVER_URL", value=FEAST_SERVER_URL),
                    ],
                    resources=V1ResourceRequirements(
                        requests={"cpu": "500m", "memory": "1Gi"},
                        limits={"cpu": "2", "memory": "4Gi"}
                    ),
                    readiness_probe=V1Probe(
                        http_get=V1HTTPGetAction(path="/v2/health/ready", port=8080),
                        initial_delay_seconds=60,
                        period_seconds=10
                    ),
                    volume_mounts=[
                        V1VolumeMount(name="scripts", mount_path="/scripts"),
                        V1VolumeMount(name="model-storage", mount_path="/shared"),
                    ]
                )
            ],
            volumes=[
                V1Volume(name="scripts", config_map=V1ConfigMapVolumeSource(name=CONFIGMAP_NAME)),
                V1Volume(name="model-storage", persistent_volume_claim=V1PersistentVolumeClaimVolumeSource(claim_name=PVC_NAME)),
            ]
        )
    )
)

print("✅ InferenceService spec created")

In [ ]:
# Deploy InferenceService
try:
    existing = kserve_client.get(MODEL_NAME, namespace=NAMESPACE)
    print(f"⚠️ InferenceService '{MODEL_NAME}' exists, replacing...")
    kserve_client.replace(MODEL_NAME, isvc, namespace=NAMESPACE)
except Exception:
    print(f"📦 Creating InferenceService '{MODEL_NAME}'...")
    kserve_client.create(isvc)

print(f"✅ InferenceService submitted to namespace '{NAMESPACE}'")

In [ ]:
# Wait for ready
print("⏳ Waiting for InferenceService to be ready...")
print("   This may take 1-2 minutes for the first deployment.")

try:
    kserve_client.wait_isvc_ready(MODEL_NAME, namespace=NAMESPACE, timeout_seconds=300)
    isvc_status = kserve_client.get(MODEL_NAME, namespace=NAMESPACE)
    url = isvc_status.get("status", {}).get("url", "")
    print(f"✅ InferenceService ready!")
    print(f"   URL: {url}")
except Exception as e:
    print(f"⚠️ Timeout waiting for ready: {e}")
    print("   Check pod status: kubectl get pods -n feast-trainer-demo -l app=sales-forecasting")

---
## Step 4: Make Predictions (V2 Protocol)

### V2 Inference Protocol

KServe uses the V2 inference protocol. Request format:

```json
{
  "inputs": [
    {
      "name": "entities",
      "shape": [n],
      "datatype": "BYTES",
      "data": [{"store_id": 1, "dept_id": 3}, ...]
    }
  ]
}
```

Response format:

```json
{
  "outputs": [
    {
      "name": "predictions",
      "shape": [n],
      "datatype": "FP32",
      "data": [96763.45, ...]
    }
  ]
}
```

In [ ]:
# Construct endpoint URL
ENDPOINT = f"http://{MODEL_NAME}-predictor.{NAMESPACE}.svc.cluster.local:8080"

# Check health
try:
    resp = requests.get(f"{ENDPOINT}/v2/health/ready", timeout=5)
    ready = resp.status_code == 200
except:
    ready = False
    
print(f"Inference Endpoint: {ENDPOINT}")
print(f"Ready: {'✅ Yes' if ready else '❌ No'}")

In [ ]:
def predict_with_feast(entities: list) -> dict:
    """Make prediction using KServe V2 protocol with Feast feature lookup.
    
    Args:
        entities: List of entity dicts, e.g., [{"store_id": 1, "dept_id": 3}]
    
    Returns:
        Dict with predictions, latency, and entities
    """
    url = f"{ENDPOINT}/v2/models/{MODEL_NAME}/infer"
    payload = {
        "inputs": [
            {
                "name": "entities",
                "shape": [len(entities)],
                "datatype": "BYTES",
                "data": entities
            }
        ]
    }
    
    t0 = time.time()
    response = requests.post(url, json=payload, timeout=30)
    latency = (time.time() - t0) * 1000
    
    if response.status_code != 200:
        raise Exception(f"Prediction failed: {response.text}")
    
    result = response.json()
    predictions = result.get("outputs", [{}])[0].get("data", [])
    
    return {
        "predictions": predictions,
        "latency_ms": latency,
        "entities": entities
    }

In [ ]:
# Single prediction
print("📊 Single Prediction Test")
print("-" * 50)

entity = {"store_id": 1, "dept_id": 3}
try:
    result = predict_with_feast([entity])
    print(f"✅ Store {entity['store_id']}, Dept {entity['dept_id']}")
    print(f"   Predicted Weekly Sales: ${result['predictions'][0]:,.0f}")
    print(f"   Latency: {result['latency_ms']:.0f}ms")
except Exception as e:
    print(f"❌ Prediction error: {e}")

In [ ]:
# Batch predictions
print("📊 Batch Prediction Test")
print("-" * 50)

entities = [
    {"store_id": s, "dept_id": d}
    for s in [1, 10, 25, 45]
    for d in [1, 5, 10, 14]
]
print(f"Scoring {len(entities)} entities...")

try:
    result = predict_with_feast(entities)
    results_df = pd.DataFrame([
        {**e, "prediction": f"${p:,.0f}"}
        for e, p in zip(entities, result["predictions"])
    ])
    print(f"\n✅ {len(results_df)} predictions in {result['latency_ms']:.0f}ms")
    print(f"   Avg latency per prediction: {result['latency_ms']/len(entities):.1f}ms")
    print()
    display(results_df)
except Exception as e:
    print(f"❌ Batch prediction error: {e}")

---
## ✅ Pipeline Complete!

### End-to-End Summary

| Notebook | Component | Output |
|----------|-----------|--------|
| 01 - Features | Feast | Features registered, online store populated |
| 02 - Training | PyTorch + Ray | Model trained with `get_historical_features()` |
| 03 - Inference | KServe | Model serving with Feast integration |

### Key Benefits

| Benefit | How |
|---------|----- |
| **No feature skew** | Same FeatureService (`inference_features`) for train & serve |
| **Scalable training** | Ray distributes `get_historical_features()` |
| **Scalable serving** | KServe auto-scales 1-3 replicas |
| **Low latency** | Feast online store (<50ms feature lookup) |
| **Simple client** | Send entity IDs only, get predictions |

### Troubleshooting

| Issue | Solution |
|-------|----------|
| InferenceService not ready | Check pod logs: `kubectl logs -l app=sales-forecasting -n feast-trainer-demo` |
| Feast lookup fails | Verify Feast server: `kubectl get pods -l app=feast-server -n feast-trainer-demo` |
| Model load error | Check model artifacts exist in `/shared/models/` |
| RBAC error (403) | Ensure ServiceAccount has ConfigMap access |

### Production Deployment

For production, use the manifest-based approach:

```bash
# Deploy all infrastructure
kubectl apply -k manifests/

# Monitor training
kubectl logs -f -l trainer.kubeflow.org/trainjob-name=sales-training

# Check serving
kubectl get inferenceservice -n feast-trainer-demo
```